In this tutorial, we'll show sending streaming data to a chart.  This assumes that you have gone through the **chart_basics** Notebook tutorial, and know how to create a chart in Galyleo and view it on the web.  First, let's build a class that will generate a sine wave and cosine wave incrementally.

In [ ]:
import math
class Waves:
    def __init__(self, init_x = 0, increment_x = 0.1):
        self.init_x = init_x
        self.increment = increment_x
        self.reset()
    
    def reset(self):
        self.x = self.init_x
    
    def next_point(self):
        result = [self.x, math.sin(self.x), math.cos(self.x)]
        self.x += self.increment
        return result
    
    

Display the chart.  This time we'll do an AreaChart.

In [ ]:
from lively_client import public_client
from IPython.display import IFrame

my_client = public_client.LivelyClient()
room_url = my_client.get_unique_room_url(chart_type="AreaChart", my_chart_id=None, room_name=None)
IFrame(room_url, width=700, height=400)

Now create a Waves object, and get the first few points

In [ ]:
from IPython.display import HTML
h = HTML('<a href=%s>%s</a>'% (room_url, room_url))
display(h)

In [ ]:
w = Waves()
data = [w.next_point() for i in range(20)]
data.insert(0, ['x', 'sin', 'cos'])

In [ ]:
data

Now send the data to the chart.  Note the **streaming** element of the dictionary that's the first argument to **send_data**.  This tells the client that future updates **append** to rather than replace the data.  The **streaming** object has a maximum of two entries: **setup**, which if **True** tells the client to **replace** rather than **append to** previous data.  **maxElements** gives the maximum number of points to be plotted; subsequent sends will roll previous entries off the list.

In [ ]:
from IPython.display import HTML
h = HTML('<a href=%s>%s</a>'% (room_url, room_url))
display(h)

In [ ]:
chart_dictionary = {"data": data, "streaming": {"setup": True, "maxElements": 30}}
ret_message, room, ret_code = my_client.send_data(chart_dictionary, chart_type="AreaChart", room_name=my_client.room_name)

Now that that is done, let's send 20 more points to the chart.  Note that only the last 30 are kept

In [ ]:
new_data = [w.next_point() for i in range(20)]
my_client.send_data({"data": new_data, "streaming": {"setup": False}}, chart_type="AreaChart", room_name=my_client.room_name)

Now let's send a point a second and watch the chart change.

In [ ]:
import time
for i in range(100):
    my_client.send_data({"data": [w.next_point()], "streaming": {"setup": False}}, chart_type = "AreaChart", room_name = my_client.room_name)
    time.sleep(1)
    